In [3]:
import pandas as pd

In [5]:
unpickled_df = pd.read_pickle("./datasets_new/dataset.pkl")  

In [7]:
unpickled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170554590 entries, 4346592 to 156191136
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   cve         object        
 1   epss        float64       
 2   date        datetime64[ns]
 3   percentile  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 6.4+ GB
